# Categorical Search

<a href="https://colab.research.google.com/github/myscale/examples/blob/main/categorical-search.ipynb" style="padding-left: 0.5rem;"><img src="https://colab.research.google.com/assets/colab-badge.svg?style=plastic)](https://colab.research.google.com/github/myscale/examples/blob/main/categorical-search.ipynb)"></a>
<a href="https://github.com/myscale/examples/blob/main/categorical-search.ipynb" style="padding-left: 0.5rem;"><img src="https://img.shields.io/badge/Open-Github-blue.svg?logo=github&style=plastic)](https://github.com/myscale/examples/blob/main/categorical-search.ipynb)"></a>

## Introduction

This notebook will show you how to access your MyScale cluster, create table, import data, and execute SQL queries in just a few simple steps.


## Prerequisites
Before getting started with python, you need to install [Clickhouse client](https://clickhouse.com/docs/en/integrations/language-clients/python/intro/).

In [1]:
!pip install clickhouse-connect

## Access MyScale cluster

To connect to your cluster with python, you'll need to provide your cluster host, username, and password. Follow the description in [Connection Details](https://docs.myscale.com/en/cluster-management/#connection-details) to get your connection infomation.

In [1]:
import clickhouse_connect

# initialize client
# note that you can retrieve your CLUSTER_HOST from your CLUSTER_URL, formatted as "https://{HOST}:{PORT}"
client = clickhouse_connect.get_client(host='YOUR_CLUSTER_HOST', port=8443, username='YOUR_USERNAME', password='YOUR_CLUSTER_PASSWORD')

## Importing data

To import data into MyScale, follow these three steps:

1. Create a table
2. Insert data into the table
3. Build a vector index

### Creating a table
Tables must be created in MyScale before you can import data.

Create a new table named `default.myscale_categorical_search`.

In [2]:
# Create a table with 128 dimensional vectors.
client.command("""
CREATE TABLE default.myscale_categorical_search
(
    id    UInt32,
    data  Array(Float32),
    CONSTRAINT check_length CHECK length(data) = 128,
    date  Date,
    label Enum8('person' = 1, 'building' = 2, 'animal' = 3)
)
ORDER BY id""")

''

### Inserting data
MyScale currently supports data import from AWS S3 and other cloud services with S3-compatible APIs, such as Tencent Cloud Object Service.

Use SQL to import data into the `default.myscale_categorical_search` table. Supported file formats include `CSV`, `CSVWithNames`, `JSONEachRow`, and `Parquet`. Please see [Formats for Input and Output Data](https://clickhouse.com/docs/en/interfaces/formats/) for detailed description of all supported formats.

In [3]:
client.command("""
INSERT INTO default.myscale_categorical_search
    SELECT * FROM s3(
        'https://d3lhz231q7ogjd.cloudfront.net/sample-datasets/quick-start/categorical-search.csv',
        'AKIDIQI6evvf8LHxMTJtDF0UifLktlLegRv8', 'xe3B2cvxiCRP5UdU5PTGlzw71e06uSji',
        'CSVWithNames',
        'id UInt32, data Array(Float32), date Date, label Enum8(''person'' = 1, ''building'' = 2, ''animal'' = 3)'
    )""")


''

### Building a vector index
In addition to creating traditional indexes on structured data, you can also create a Vector Index in MyScale for vector embeddings.

**Step 1**. Create an MSTG vector index and check vector index build status

The build time of the index will depend on the size of your data import

In [5]:
client.command("""
ALTER TABLE default.myscale_categorical_search
    ADD VECTOR INDEX categorical_vector_idx data
    TYPE MSTG""")

''

**Step 2**. Use SQL to check vector index build status

In [8]:
# Query the 'vector_indices' system table to check the status of the index creation.
get_index_status="SELECT status FROM system.vector_indices WHERE table='myscale_categorical_search'"

# Print the status of the index creation.  The status will be 'Built' if the index was created successfully.
print(f"index build status is {client.command(get_index_status)}")

index build status is Built


## Executing SQL queries
After importing data into a MyScale table, queries may be performed on the data within the table. In no time, you’ll be able to experience the quick query speeds attainable with MyScale.

### Vector search
Conventionally, queries are made using text or images, such as “a blue car” or images of a blue car. However, MyScale treats all queries as vectors, and gives a response to the query based on the similarity (“distance“) between the query and the existing data within the table.

To retrieve data using a vector as the query, use this SQL command as below, which returns the 10 most similar results

In [9]:
# pick a random row from the table as the target
random_row = client.query("SELECT * FROM default.myscale_categorical_search ORDER BY rand() LIMIT 1")
assert random_row.row_count == 1
target_row_id = random_row.first_item["id"]
target_row_label = random_row.first_item["label"]
target_row_date = random_row.first_item["date"]
target_row_data = random_row.first_item["data"]
print("currently selected item id={}, label={}, date={}".format(target_row_id, target_row_label, target_row_date))

# Fetch the result of the query.
result = client.query(f"""
SELECT id, date, label, 
    distance(data, {target_row_data}) as dist 
FROM default.myscale_categorical_search
ORDER BY dist ASC
LIMIT 10
""")

# Iterate through the rows of the query result and print the 'id', 'date',
# 'label', and distance for each row.
print("Top 10 candidates:")
for row in result.named_results():
    print(row["id"], row["date"], row["label"], row["dist"])

currently selected item id=789292, label=building, date=1985-12-18
Top 10 candidates:
789292 1985-12-18 building 0.0
407891 2020-11-17 person 50215.0
790816 1993-06-28 building 51787.0
402999 1977-09-09 building 58818.0
10452 2031-03-10 person 59212.0
203989 2031-01-06 animal 59496.0
204908 1977-01-05 building 62080.0
590194 2008-11-30 building 62286.0
802112 1997-02-15 person 65755.0
394816 1977-06-22 animal 66460.0


### Filtered search
In the earlier demonstration, a query was executed using vector embeddings. Additionally, users can also perform queries that utilize a combination of structured and vector data in SQL.

In [10]:
# Fetch the result of the query.
result = client.query(f"""
SELECT id, date, label, 
    distance(data, {target_row_data}) as dist
FROM default.myscale_categorical_search
WHERE toYear(date) >= 2000 AND label = 'animal'
ORDER BY dist ASC
LIMIT 10
""")

# Iterate through the rows of the query result and print the 'id', 'date',
# 'label', and distance for each row.
for row in result.named_results():
    print(row["id"], row["date"], row["label"], row["dist"])

203989 2031-01-06 animal 59496.0
787606 2027-02-23 animal 68521.0
602833 2032-07-04 animal 73845.0
610401 2018-10-13 animal 74414.0
7547 2026-11-26 animal 76731.0
407873 2033-02-24 animal 81271.0
202149 2025-09-17 animal 82537.0
798320 2006-07-13 animal 84778.0
393951 2029-04-21 animal 86319.0
399191 2018-07-31 animal 88445.0
